In [5]:
from UNet import UNet
from ScanDataset import ScanDataset
from utils import *

import os
from random import shuffle

from torch.autograd import Variable
import torch.utils.data as data
import torch

import math
import sys

use_cuda = torch.cuda.is_available()
dtype = torch.cuda.FloatTensor if use_cuda else torch.FloatTensor

%matplotlib inline

%load_ext autoreload
%autoreload 2
%reload_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
use_cuda = torch.cuda.is_available()
dtype = torch.cuda.FloatTensor if use_cuda else torch.FloatTensor

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

In [3]:
luna_root = '/deep/group/shubhang/luna16/'

def train(model, data_loader, num_epochs=1, losses_file=None, ckpt_file=None, resume=False):
    losses = list(np.load(losses_file)) if resume else []
    num_batches = len(data_loader)
    for epoch in range(num_epochs):
        print("Epoch %d" % (epoch+1))
        for i, batch in enumerate(data_loader):
            X, y = batch
            
            loss = model.train(Variable(X.type(dtype)), Variable(y.type(dtype)))
            losses.append(loss.data.cpu()[0])

            progress = math.ceil((i+1)/num_batches*20)
            sys.stdout.write('\r[' + progress * '*'+ (20 - progress) * ' ' + '] (%d/%d) Loss: %f' % ((i+1), num_batches, loss.data[0]))

            if i % 10 == 0:
                model.save(ckpt_file)
                np.save(losses_file, losses)
        
        print('\n')

In [4]:
resume_name = 'train10'
unet = UNet(lr=1e-2, pos_weight=5000)
unet.load('ckpts/' + resume_name + '_ckpt')
scan_dataset = ScanDataset(luna_root + 'lungs_slices/train/', luna_root + 'gt_slices/train/')
scan_loader = data.DataLoader(scan_dataset, batch_size=40, shuffle=True, **kwargs)

In [6]:
experiment_name = 'train11'
losses_file = 'losses/' + experiment_name + '.npy'
train(unet, scan_loader, num_epochs=1, losses_file=losses_file, ckpt_file=experiment_name + '_ckpt', resume=False)

Epoch 1
[********************] (4324/4324) Loss: 0.047634

